In [129]:
from ows_language_model.config import config

INTERIM_PATH = config.DATASET_DIR / 'interim'

with open(INTERIM_PATH / 'clean_docs.json', 'r') as f:
    english_docs = json.load(f)
    

In [130]:
! pip install --upgrade fastai torch sentencepiece

Requirement already up-to-date: fastai in /Users/henrymaguire/Work/side-projects/one-word-story/ows_language_model/venv/lib/python3.7/site-packages (2.1.2)
Requirement already up-to-date: torch in /Users/henrymaguire/Work/side-projects/one-word-story/ows_language_model/venv/lib/python3.7/site-packages (1.7.0)
Requirement already up-to-date: sentencepiece in /Users/henrymaguire/Work/side-projects/one-word-story/ows_language_model/venv/lib/python3.7/site-packages (0.1.94)


You are using pip version 19.0.3, however version 20.2.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [131]:
from fastai.text.all import (untar_data, URLs, get_text_files, WordTokenizer, 
                             first, coll_repr, Tokenizer, SubwordTokenizer,
                             Numericalize, L, LMDataLoader, partial, DataBlock, 
                             TextBlock, RandomSplitter, AWD_LSTM, accuracy, Perplexity,
                             language_model_learner)

In [133]:
path = untar_data(URLs.IMDB)
files = get_text_files(path, folders=['train', 'test', 'unsup'])
#txt = files[0].open().read(); txt[:75]

spacy = WordTokenizer()
tkn = Tokenizer(spacy)

In [134]:
txts = L(o.open().read() for o in files[:2000])

In [135]:
print(coll_repr(toks, 30))
print(first(spacy(['Hello this is a U.S. dollar bill'])))

(#121) ['Alan','Rickman','&','Emma','Thompson','give','good','performances','with','southern','/','New','Orleans','accents','in','this','detective','flick','.','It',"'s",'worth','seeing','for','their','scenes-','and','Rickman',"'s",'scene'...]
['Hello', 'this', 'is', 'a', 'U.S.', 'dollar', 'bill']


In [136]:
print(coll_repr(tkn(txt), 31))

(#139) ['xxbos','xxmaj','alan','xxmaj','rickman','&','xxmaj','emma','xxmaj','thompson','give','good','performances','with','southern','/','xxmaj','new','xxmaj','orleans','accents','in','this','detective','flick','.','xxmaj','it',"'s",'worth','seeing'...]


In [137]:
toks200 = txts[:200].map(tkn)

In [139]:
num = Numericalize(min_freq=2, max_vocab=60000)
num.setup(toks200)
print(len(num.vocab))
coll_repr(num.vocab, 20)
nums200 = toks200.map(num)

3120


In [89]:
print(' '.join(num.vocab[o] for o in x[0][:20]))
print(' '.join(num.vocab[o] for o in y[0][:20]))

xxbos xxmaj xxunk xxmaj rickman & xxmaj xxunk xxmaj thompson give good performances with southern / xxmaj new xxmaj xxunk
xxmaj xxunk xxmaj rickman & xxmaj xxunk xxmaj thompson give good performances with southern / xxmaj new xxmaj xxunk accents


In [140]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dl = LMDataLoader(nums200)

In [141]:
dls_lm = DataBlock(blocks=TextBlock.from_folder(path, is_lm=True),
                   get_items=get_imdb, splitter=RandomSplitter(0.1)).dataloaders(
                path, path=path, bs=128, seq_len=80)

In [121]:
dls_lm.show_batch(max_n=20)

,text,text_
0,"xxbos xxmaj this is one of xxmaj bruce xxmaj willis ' finest films yet . xxmaj he plays a tired old cop who is handed the menial task of driving xxmaj mos xxmaj def , a criminal , who has to testify in court . xxmaj little does he know , he winds up running into very much trouble . xxmaj although one might think that this movie can not be suspenseful because it takes place within the radius of","xxmaj this is one of xxmaj bruce xxmaj willis ' finest films yet . xxmaj he plays a tired old cop who is handed the menial task of driving xxmaj mos xxmaj def , a criminal , who has to testify in court . xxmaj little does he know , he winds up running into very much trouble . xxmaj although one might think that this movie can not be suspenseful because it takes place within the radius of 16"
1,"the good doc a crazed xxunk and all this time big xxunk xxunk on a table awaiting his electricity fix so he can wreak some xxunk was kind of a short movie , around 70 minutes and some change , but the action is there , and the great actors are there as xxunk atwill turns up as a police inspector , heres some trivia , lionel atwill appeared in son of xxmaj frankenstein , ghost of frankenstein , frankenstein","good doc a crazed xxunk and all this time big xxunk xxunk on a table awaiting his electricity fix so he can wreak some xxunk was kind of a short movie , around 70 minutes and some change , but the action is there , and the great actors are there as xxunk atwill turns up as a police inspector , heres some trivia , lionel atwill appeared in son of xxmaj frankenstein , ghost of frankenstein , frankenstein meets"
2,"new friends , because they seem to be cool and in harmony with their nature . xxmaj he also fell in love with female alien , xxmaj xxunk - xxunk , er … xxmaj xxunk . xxmaj battle between humans and aliens . xxmaj the end . -- xxmaj well , not only has been this around thousand times over and over again , but it ( go figure ! ) might have taken some bits from the history as","friends , because they seem to be cool and in harmony with their nature . xxmaj he also fell in love with female alien , xxmaj xxunk - xxunk , er … xxmaj xxunk . xxmaj battle between humans and aliens . xxmaj the end . -- xxmaj well , not only has been this around thousand times over and over again , but it ( go figure ! ) might have taken some bits from the history as well"
3,"with 4 of my friends , and towards the end i could not stop squeezing my friend 's hand . i was in a state of just * awe * for 1 + days after seeing it . xxmaj it made me feel a lot like how i felt after "" escape from xxmaj xxunk . "" i predicted how the ending would play out ( and i was right ) even though i did not want it to .","4 of my friends , and towards the end i could not stop squeezing my friend 's hand . i was in a state of just * awe * for 1 + days after seeing it . xxmaj it made me feel a lot like how i felt after "" escape from xxmaj xxunk . "" i predicted how the ending would play out ( and i was right ) even though i did not want it to . xxmaj"
4,"tough , experienced copper ? xxmaj doubtful ▁ xxmaj perhaps what annoyed me most about the cast ( shallow , i admit ) is seeing how one of my all - time favorite xxmaj b - actors , xxmaj jeffrey xxmaj combs , is given such a small and insignificant role ! xxmaj that 's xxmaj doctor xxmaj herbert xxmaj west from re - animator we 're talking about , dammit ! xxmaj show a little more respect , xxmaj",", experienced copper ? xxmaj doubtful ▁ xxmaj perhaps what annoyed me most about the cast ( shallow , i admit ) is seeing how one of my all - time favorite xxmaj b - actors , xxmaj jeffrey xxmaj combs , is given such a small and insignificant role ! xxmaj that 's xxmaj doctor xxmaj herbert xxmaj west from re - animator we 're talking about , dammit ! xxmaj show a little more respect , xxmaj malone"
5,"do n't blame the cast for making this less than engaging , i blame the writer who is constantly changing who the principl

In [142]:
learn = language_model_learner(dls_lm, 
                               AWD_LSTM, drop_mult=0.3,
                               metrics=[accuracy, Perplexity()])#.to_fp16()

In [143]:
learn.fit_one_cycle(1, 2e-2)

RuntimeError: The size of tensor a (400) must match the size of tensor b (0) at non-singleton dimension 2

In [120]:
for x in dls_lm:
    print(x)